<a href="https://colab.research.google.com/github/Tim-Sa/text_diff_exp_notebook/blob/main/morph_stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
%%capture
!pip install natasha
import natasha
from razdel import sentenize, tokenize
from ipymarkup import show_dep_ascii_markup as show_markup
from navec import Navec
from slovnet import Morph, Syntax
from slovnet.markup import MorphMarkup, SyntaxMarkup

import aiohttp

import re
from pprint import pprint
from collections import Counter
from dataclasses import dataclass

Установка и инициализация моделей синтаксического и морфологического анализа.
[Проект Natasha](https://github.com/natasha)

In [117]:
async def a_download_by_url(
    url: str,
    target_file_path: str,
    chunk_size: int = 1024
) -> None:

    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            with open(target_file_path, "wb") as handle:
                async for data in response.content.iter_chunked(chunk_size):
                    handle.write(data)


await a_download_by_url('https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar', '/content/navec_news_v1_1B_250K_300d_100q.tar')
await a_download_by_url('https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_morph_news_v1.tar',         '/content/slovnet_morph_news_v1.tar')
await a_download_by_url('https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_syntax_news_v1.tar',        '/content/slovnet_syntax_news_v1.tar')

navec  = Navec.load('/content/navec_news_v1_1B_250K_300d_100q.tar')
morph  = Morph.load('/content/slovnet_morph_news_v1.tar', batch_size=4)
syntax = Syntax.load('/content/slovnet_syntax_news_v1.tar')

morph.navec(navec)
syntax.navec(navec)
pass

In [116]:
@dataclass(eq = True, frozen = True)
class Word:
    '''
    Класс для отображения информации о слове.

    Attributes:
        id (int): Индекс слова в предложении.
        word (str): Строковое представление слова.
        morph (str): Морфологический тип слова (например: ADV, NOUN или PUNCT).
    '''
    id: int
    word: str
    morph: str

    def __len__(self):
        return len(self.word)

    def __str__(self):
        return self.word

    def ljust(self, size: int):
        return self.word.ljust(size)

@dataclass
class Collocation:
    '''
    Класс для отображения коллокации двух слов (слова и знака пунктуации) с указанием типа связи.

    Attributes:
        source_word (Word): Слово, которое определяет связь.
        target_word (Word): Слово, являющееся объетом связи.
        rel (str): Тип связи между source_word и target_word, (например: conj, case, iobj, punct и т.д.)
    '''
    source_word: Word
    target_word: Word
    rel: str

In [115]:
def collocations(text: str) -> list[Collocation]:
    '''
    Разбивает исходный текст на коллокации (словосочетания, имеющее признаки синтаксически и семантически целостной единицы)

    Args:
        text (str): Текст с пунктуацией и сохраненным регистром.

    Returns:
        collocations (list[Collocation]): список коллокаций (каждая коллокация представлена парой слов и типом связи)
    '''

    tokens = tokenize(text)
    chunk = [[_.text for _ in tokens]]

    syntax_markup = syntax.map(chunk)
    morph_markup = morph.map(chunk)

    collocations = list()

    morph_tags = dict()
    for token in next(morph_markup).tokens:

        tag_name_end = token.tag.find('|')
        token_tag = token.tag[:tag_name_end]

        if not token.text in morph_tags:
            morph_tags[token.text] = token_tag

    for token in next(syntax_markup).tokens:
        source = int(token.head_id) - 1
        target = int(token.id) - 1

        source_word_text = chunk[0][source]
        target_word_text = chunk[0][target]

        if source > 0 and source != target:  # skip root, loops

            source_morph = morph_tags[source_word_text]
            target_morph = morph_tags[target_word_text]

            source_word = Word(
                id=source,
                word=source_word_text,
                morph=source_morph
            )

            target_word = Word(
                id=target,
                word=target_word_text,
                morph=target_morph
            )

            collocation = Collocation(
                source_word=source_word,
                target_word=target_word,
                rel = token.rel
            )

            collocations.append(collocation)

    return collocations

In [114]:
def _restore_words_order(collocations: list[Collocation]) -> list[str]:
    used_id = []
    words = []

    for colloc in collocations:
        words.append(colloc.source_word)
        words.append(colloc.target_word)

    sorted_words = sorted(words, key=lambda word: word.id)

    ordered_words = []
    prev_id = None
    for word in sorted_words:
        if word.id != prev_id:
            ordered_words.append(word)
            prev_id = word.id

    return ordered_words

In [113]:
def show_collocations_deps(
    collocations: list[Collocation],
    ignore: list[str] = []
) -> None:
    '''
    Отображает в stdout схему

    Args:
        collocations (list[Collocation]): список коллокаций (каждая коллокация представлена парой слов и типом связи)
    '''

    words = _restore_words_order(collocations)

    deps = []
    for colloc in collocations:
        if colloc.rel not in ignore:

            deps.append(
                [colloc.source_word.id,
                 colloc.target_word.id, colloc.rel]
            )
    show_markup(words, deps)

In [112]:
def filter_collocations(
    collocations: list[Collocation],
    rels_to_remove: list[str]
) -> list[Collocation]:

    def good_collocation(collocation: Collocation) -> bool:
        return collocation.rel not in rels_to_remove

    filtered = filter(good_collocation,
                      collocations)

    return list(filtered)

In [111]:
def prune_text(text: str, rejected_rels: list[str], show_steps: bool = False) -> str:
    '''
    Убирает из текста те слова, которые находятся в фильтруемых коллокациях.
    Коллокации фильтруются по синтаксическим отношениям между двумя словами коллокации.

    Args:
        text (str): Текст с пунктуацией и сохраненным регистром.
        rejected_rels (list[str]): список типов синтаксических отношений, которые нужно ислючить из текста (['punct', 'discourse'])
        (Optional) show_steps (bool): Выводить ли в stdout выделенные коллокации и отфильтрованные слова.
    Returns:
        text (str): отфильтрованный текст с сохранением исходного порядка слов.
    '''

    colls = filter_collocations(collocations(text), rejected_rels)

    if show_steps:
        pprint(colls, indent=4)

    words_order = _restore_words_order(colls)

    if show_steps:
        pprint(words_order, indent=4)

    words = list(map(str, words_order))

    return " ".join(words)

In [110]:
def get_rels_list(collocations: list) -> list:
    rels = list()
    for col in collocations:
        rels.append(col.rel)
    return rels

In [109]:
def count_rels(text: str) -> dict:
  colls = collocations(text)
  rels = get_rels_list(colls)
  return dict(Counter(rels))

In [108]:
@dataclass
class TextStat:
    text: str

    def __post_init__(self):
        self.nb = self.count_letters()
        self.ns = self.count_words()
        self.np = self.count_sentences()
        self.colls = self.count_collocations()

    def count_letters(self) -> int:
        """Подсчитывает количество букв в тексте."""
        return sum(1 for char in self.text if char.isalpha())

    def count_words(self) -> int:
        """Подсчитывает количество слов в тексте."""
        return len(self.text.split())

    def count_sentences(self) -> int:
        """Подсчитывает количество предложений в тексте."""
        predicates = re.split(r'[.!?]+', self.text.strip())
        return len([p for p in predicates if p.strip()])  # Учитываем только непустые предложения

    def count_collocations(self) -> int:
        """Подсчитывает количество семантических отношений."""
        return len(collocations(self.text))

    def average_word_length(self, n_round: int = 2) -> float:
        """Возвращает среднюю длину слова в тексте."""
        return round(self.nb / self.ns, n_round) if self.ns > 0 else 0.0

    def average_sentence_length(self, n_round: int = 2) -> float:
        """Возвращает среднюю длину предложения в тексте."""
        return round(self.ns / self.np, n_round) if self.np > 0 else 0.0

    def letters_per_collocation(self, n_round: int = 2) -> float:
        """Возвращает количество букв на семантическое отношение."""
        return round(self.nb / self.colls, n_round) if self.colls > 0 else 0.0

    def collocations_per_word(self, n_round: int = 2) -> float:
        """Возвращает количество семантических отношений на слово."""
        return round(self.colls / self.ns, n_round) if self.ns > 0 else 0.0

    def collocations_per_sentence(self, n_round: int = 2) -> float:
        """Возвращает количество семантических отношений на предложение."""
        return round(self.colls / self.np, n_round) if self.np > 0 else 0.0

    def __str__(self):
        return (f"Количество букв в тексте: {self.nb}\n"
                f"Количество слов в тексте: {self.ns}\n"
                f"Количество предложений: {self.np}\n"
                f"Ср. длина слова: {self.average_word_length()}\n"
                f"Ср. длина предложения: {self.average_sentence_length()}\n"
                f"Кол-во сем. отношений: {self.colls}\n"
                f"Букв на сем. отношение: {self.letters_per_collocation()}\n"
                f"Сем. отношений на слово: {self.collocations_per_word()}\n"
                f"Сем. отношений на предложение: {self.collocations_per_sentence()}")

In [107]:
text = """
Одним из внезапных итогов Петербургского международного экономического форума 2024 года
стало практическое отсутствие ставших уже привычными громких заявлений о намерении
произвести в ближайшие годы все новые сотни гражданских самолетов.
Сообщение о запуске авиашаттла между Москвой и Санкт-Петербургом лишь незначительно компенсировало эту оглушающую тишину.
Конечно, «громадье планов» неумолимо сокращалось по мере столкновения журналистов, социологов, экономистов и прочих пиарщиков,
которым поручили составлять и реализовывать эти планы, с загадочной для них технологической реальностью.
Однако полное прекращение радостных сообщений о том, что единственный экземпляр очередного самолета теперь надо официально именовать «серийным», вызвало интерес и привлекло к себе внимание.
Причина проста: внезапно оказалось, что за весь 2023 год российское гражданское авиастроение,
крайне успешно и эффектно «поднимаемое с колен» теми же самыми эффективными менеджерами, которые ставили его на эти колени долгие годы, произвело аж 9 самолетов.
"""

text_stat = TextStat(text)

rels_list = set(get_rels_list(collocations(text)))
rels_count = count_rels(text)

print(text_stat)

print('\nСписок синтакс. отношений, извлеченных из текста:')
pprint(rels_list, indent=4)

print('\nЧастота представленности синтакс. отношений, извлеченных из текста:')
pprint(rels_count, indent=4)

Количество букв в тексте: 875
Количество слов в тексте: 130
Количество предложений: 5
Ср. длина слова: 6.73
Ср. длина предложения: 26.0
Кол-во сем. отношений: 148
Букв на сем. отношение: 5.91
Сем. отношений на слово: 1.14
Сем. отношений на предложение: 29.6

Список синтакс. отношений, извлеченных из текста:
{   'acl',
    'acl:relcl',
    'advmod',
    'amod',
    'case',
    'cc',
    'ccomp',
    'conj',
    'csubj',
    'det',
    'fixed',
    'iobj',
    'mark',
    'nmod',
    'nsubj',
    'nummod',
    'obj',
    'obl',
    'parataxis',
    'punct',
    'xcomp'}

Частота представленности синтакс. отношений, извлеченных из текста:
{   'acl': 5,
    'acl:relcl': 2,
    'advmod': 12,
    'amod': 24,
    'case': 13,
    'cc': 5,
    'ccomp': 2,
    'conj': 7,
    'csubj': 1,
    'det': 6,
    'fixed': 1,
    'iobj': 1,
    'mark': 2,
    'nmod': 12,
    'nsubj': 8,
    'nummod': 2,
    'obj': 7,
    'obl': 10,
    'parataxis': 3,
    'punct': 23,
    'xcomp': 2}
